# Plot polygons over SKR Flux
This notebook demonstrates how to plot polygons on top of an SKR spectrogram for a specific time frame.

In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mp_colors
import matplotlib.dates as mdates
from matplotlib.patches import Polygon

from tfcat import TFCat

from lfe_func import get_sav_data

Define file paths

In [2]:
file_catalogue = '../data/calculated/2004001_2017258_joint_dst_catalogue.json'

Read Sav file

In [3]:
year = 2012
time, frequency, flux = get_sav_data(year)

Import Polygon coordinates

In [4]:
catalogue = TFCat.from_file(file_catalogue)

poly_coordinates = []
for i in range(len(catalogue._data['features'])):
    poly_time, poly_freq = np.array(catalogue._data['features'][i]['geometry']['coordinates'][0]).T

    poly_time = poly_time.astype('datetime64[s]')

    poly_coords = np.array([mdates.date2num(poly_time), poly_freq]).T

    poly_coordinates.append(poly_coords)

Choose upper and lower limit of time window

In [5]:
time_low, time_high = mdates.date2num(np.datetime64('2012-06-10')), mdates.date2num(np.datetime64('2012-06-14'))

Draw the Figure

In [6]:
fig = plt.figure(figsize=(20,3), dpi=100)

ax = fig.add_subplot()

norm = mp_colors.LogNorm(vmin=1e-24, vmax=1e-19)

cbar = ax.pcolormesh(time, frequency, flux, norm=norm, cmap='jet')

for poly_coords in poly_coordinates:
    
    poly_time = poly_coords[:,0]
    time_mask = (poly_time >= time_low) & (poly_time <= time_high)

    ax.add_patch(Polygon(poly_coords[time_mask], fill=False, lw=2, color='purple'))

ax.plot([], [], color='purple', label='Polygon')

fig.colorbar(cbar)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d-%H:%M'))
ax.set_xlim(np.datetime64('2012-06-10T13:08:00'), np.datetime64('2012-06-14T02:16:00'))
#ax.set_ylim(50, 500)
ax.set_yscale('log')

ax.set_title("Saturn's Magnetic Flux at set Frequency Channels with Polygons between Jun 10 and Jun 16, 2012")
ax.set_xlabel('Date')
ax.set_ylabel('Frequency (kHz)')
ax.legend(loc='lower right');